In [1]:
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import time
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select

In [4]:
import os
directory = os.getcwd() 


In [43]:
# directory

In [6]:
import requests
import zipfile
import io
import re
import subprocess
from sys import platform


class GetChromedriver:
    LATEST_URL = "http://chromedriver.storage.googleapis.com/LATEST_RELEASE"
    BASE_URL = "http://chromedriver.storage.googleapis.com/"
    MAC_FILENAME = "chromedriver_mac64.zip"
    LINUX_FILENAME = "chromedriver_linux64.zip"
    TARGET_DIR = directory  # change me

    def __init__(self):
        self.latest = self._get_latest_version()
        self.installed = self._get_installed_version()
        self.download_url = self._build_download_url()

    def _get_latest_version(self):
        response = requests.get(self.LATEST_URL)
        text = response.text
        print(f"latest version is {text}")
        return text

    def _get_installed_version(self):
        try:
            installed = str(
                subprocess.run(["chromedriver", "-v"], stdout=subprocess.PIPE).stdout
            )
        except:
            installed = (
                "0.0.0.0"
            )  # this is the pattern of the versioning chromedriver is following.
        version = re.findall("\d+\.\d+\.\d+\.\d+", installed)[0]
        print(f"installed version is {installed}")
        print(f"extracted version number is {version}")
        return version

    def _build_download_url(self):
        filename = "chromedriver_win32.zip"
        url = f"{self.BASE_URL}{self.latest}/{filename}"
        print(f"download url is {url}")
        return url

    def download_extract(self):
        if self.latest != self.installed:
            print("downloading latest chromedriver")
            response = requests.get(self.download_url, stream=True)
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))
            zip_file.extractall(self.TARGET_DIR)
            subprocess.run(["chmod", "0777", directory+"\\chromedriver.zip"])
        else:
            print("latest version is installed")


get_chromedriver = GetChromedriver()
get_chromedriver.download_extract()

latest version is 114.0.5735.90
installed version is b'ChromeDriver 114.0.5735.90 (386bc09e8f4f2e025eddae123f36f6263096ae49-refs/branch-heads/5735@{#1052})\r\n'
extracted version number is 114.0.5735.90
download url is http://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_win32.zip
latest version is installed


In [7]:
from bs4 import BeautifulSoup
import pandas as pd
import time

In [8]:
browser = webdriver.Chrome()
browser.get('https://www.columbiaasia.com/malaysia/hospitals/cheras/doctors')



In [10]:
page_source = browser.page_source
# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the select element by its ID
select_element = soup.find('select', id='edit-jump')

# Get all the options and their values
options = select_element.find_all('option')

# Extract values from the options
values = [option['value'] for option in options if 'value' in option.attrs and option['value']]

print(values)

['/malaysia/hospitals/cheras/doctors', '/malaysia/hospitals/bintulu/doctors', '/malaysia/hospitals/bukit-jalil/doctors', '/malaysia/hospitals/bukit-rimau/doctors', '/malaysia/hospitals/iskandar-puteri/doctors', '/malaysia/hospitals/klang/doctors', '/malaysia/hospitals/miri/doctors', '/malaysia/hospitals/petaling-jaya/doctors', '/malaysia/hospitals/puchong/doctors', '/malaysia/hospitals/seremban/doctors', '/malaysia/hospitals/setapak/doctors', '/malaysia/hospitals/shah-alam/doctors', '/malaysia/hospitals/taiping/doctors', '/malaysia/hospitals/tebrau/doctors']


In [29]:
values = ['/malaysia/hospitals/cheras/doctors',
 '/malaysia/hospitals/bintulu/doctors',
#  '/malaysia/hospitals/bukit-jalil/doctors', no doctor yet
 '/malaysia/hospitals/bukit-rimau/doctors',
 '/malaysia/hospitals/iskandar-puteri/doctors',
 '/malaysia/hospitals/klang/doctors',
 '/malaysia/hospitals/miri/doctors',
 '/malaysia/hospitals/petaling-jaya/doctors',
 '/malaysia/hospitals/puchong/doctors',
 '/malaysia/hospitals/seremban/doctors',
 '/malaysia/hospitals/setapak/doctors',
#  '/malaysia/hospitals/shah-alam/doctors', different format
 '/malaysia/hospitals/taiping/doctors',
 '/malaysia/hospitals/tebrau/doctors']

In [32]:
df_full = pd.DataFrame()

for i in values:
    print(i)
    browser.get("https://www.columbiaasia.com"+i)
    in_page_source = browser.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(in_page_source, 'html.parser')

    # Find the last page's link in the pagination
    last_page_link = soup.find('li', class_='pager-last').find('a')['href']

    # Extract the last page number from the link
    last_page_number = int(last_page_link.split('=')[-1])
    print("Total Pages")
    print(last_page_number)
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    # Find all the doctor entries on the page
    doctors = soup.find_all('div', class_='media-body')

    # Lists to store the extracted data
    names = []
    links = []
    speciality_titles = []
    speciality_appointments = []
    designations = []
    qualifications = []

    # Iterate through each doctor entry and extract the required information
    for doctor in doctors:
            link_element = doctor.find('a')
            name = link_element.text.strip() if link_element else None

            link = link_element['href'] if link_element else None

            speciality_element = doctor.find('div', class_='speciality-title')
            speciality_title = speciality_element.text.strip() if speciality_element else None

            appointment_element = doctor.find('a', class_='btn btn-primary')
            speciality_appointment = appointment_element['href'] if appointment_element else None

            designation_element = doctor.find('div', class_='designation')
            designation = designation_element.text.strip() if designation_element else None

            qualification_element = doctor.find('div', class_='qualification-title')
            qualification = qualification_element.text.strip() if qualification_element else None

            # Append the extracted data to the lists
            names.append(name)
            links.append(link)
            speciality_titles.append(speciality_title)
            speciality_appointments.append(speciality_appointment)
            designations.append(designation)
            qualifications.append(qualification)

    # Create a DataFrame from the lists
    data = {
        'Name': names,
        'Link': links,
        'Speciality Title': speciality_titles,
        'Speciality Appointment': speciality_appointments,
        'Designation': designations,
        'Qualification': qualifications
    }
    df = pd.DataFrame(data)
    df["link"] = i
    # Now you have the data in a DataFrame, and you can perform further operations or analysis on it

    df_full = pd.concat([df_full,df])
    time.sleep(1)
    for page_no in range(1,last_page_number+1):
        
        print("Visiting Page:"+str(page_no))
        browser.get("https://www.columbiaasia.com"+str(i)+"?page="+str(page_no))
        time.sleep(2)
        soup = BeautifulSoup(browser.page_source, 'html.parser')

        # Find all the doctor entries on the page
        doctors = soup.find_all('div', class_='media-body')

        # Lists to store the extracted data
        names = []
        links = []
        speciality_titles = []
        speciality_appointments = []
        designations = []
        qualifications = []

        # Iterate through each doctor entry and extract the required information
        for doctor in doctors:
                link_element = doctor.find('a')
                name = link_element.text.strip() if link_element else None

                link = link_element['href'] if link_element else None

                speciality_element = doctor.find('div', class_='speciality-title')
                speciality_title = speciality_element.text.strip() if speciality_element else None

                appointment_element = doctor.find('a', class_='btn btn-primary')
                speciality_appointment = appointment_element['href'] if appointment_element else None

                designation_element = doctor.find('div', class_='designation')
                designation = designation_element.text.strip() if designation_element else None

                qualification_element = doctor.find('div', class_='qualification-title')
                qualification = qualification_element.text.strip() if qualification_element else None

                # Append the extracted data to the lists
                names.append(name)
                links.append(link)
                speciality_titles.append(speciality_title)
                speciality_appointments.append(speciality_appointment)
                designations.append(designation)
                qualifications.append(qualification)

        # Create a DataFrame from the lists
        data = {
            'Name': names,
            'Link': links,
            'Speciality Title': speciality_titles,
            'Speciality Appointment': speciality_appointments,
            'Designation': designations,
            'Qualification': qualifications
        }
        df = pd.DataFrame(data)
        df["link"] = i
        # Now you have the data in a DataFrame, and you can perform further operations or analysis on it

        df_full = pd.concat([df_full,df])
        time.sleep(1)

/malaysia/hospitals/cheras/doctors
Total Pages
5
Visiting Page:1
Visiting Page:2
Visiting Page:3
Visiting Page:4
Visiting Page:5
/malaysia/hospitals/bintulu/doctors
Total Pages
3
Visiting Page:1
Visiting Page:2
Visiting Page:3
/malaysia/hospitals/bukit-rimau/doctors
Total Pages
8
Visiting Page:1
Visiting Page:2
Visiting Page:3
Visiting Page:4
Visiting Page:5
Visiting Page:6
Visiting Page:7
Visiting Page:8
/malaysia/hospitals/iskandar-puteri/doctors
Total Pages
8
Visiting Page:1
Visiting Page:2
Visiting Page:3
Visiting Page:4
Visiting Page:5
Visiting Page:6
Visiting Page:7
Visiting Page:8
/malaysia/hospitals/klang/doctors
Total Pages
6
Visiting Page:1
Visiting Page:2
Visiting Page:3
Visiting Page:4
Visiting Page:5
Visiting Page:6
/malaysia/hospitals/miri/doctors
Total Pages
3
Visiting Page:1
Visiting Page:2
Visiting Page:3
/malaysia/hospitals/petaling-jaya/doctors
Total Pages
6
Visiting Page:1
Visiting Page:2
Visiting Page:3
Visiting Page:4
Visiting Page:5
Visiting Page:6
/malaysia/hosp

In [33]:
df_full.to_csv

,Name,Link,Speciality Title,Speciality Appointment,Designation,Qualification,link
0,Dr. Adam Lau Ee Chang,doctors/dr.-adam-lau-ee-chang,General Practice,https://malaysiaportal.columbiaasia.com/webcon...,Medical Officer,MD (CSMU),/malaysia/hospitals/cheras/doctors
1,Dr. Aiffa Syafira Binti Mohamad Amir Hamzah,doctors/dr.-aiffa-syafira-binti-mohamad-amir-h...,None,None,Medical Officer,MD (UPM),/malaysia/hospitals/cheras/doctors
2,Dr. Amir Bin Ramli @ Shaari,doctors/dr.-amir-bin-ramli-@-shaari,Internal Medicine,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Internal Medicine Physician,"MD (USM), MMed (Internal Medicine) (USM)",/malaysia/hospitals/cheras/doctors
3,Dr. Arun Arunasalam A/L Poraviappan,doctors/dr.-arun-arunasalam-a~l-poraviappan,Urology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Urologist,"M Surgery (UM), FRCS (Urology) (Glasgow), Fell...",/malaysia/hospitals/cheras/doctors
4,Dr. Azfar Rizal Ahmad,doctors/dr.-azfar-rizal-ahmad,Orthopedic,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Orthopedic and Arthroscopic Shoulde...,"MBBS (Australia), MS (Orthopedics) (UKM), Fell...",/malaysia/hospitals/cheras/doctors
...,...,...,...,...,...,...,...
4,Dr. Tee Shin Kuan,doctors/dr.-tee-shin-kuan,Internal Medicine and Nephrology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Internal Medicine Physician & Nephr...,"MBBS (UM), MRCP (UK)",/malaysia/hospitals/tebrau/doctors
5,Dr. Tey Kwee Eng,doctors/dr.-tey-kwee-eng,Dermatology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Dermatologist (Visiting),"MD (UKM), MRCP (UK), Mmed (Internal Medicine)(...",/malaysia/hospitals/tebrau/doctors
0,Dr. Tharmasewary A/P Karuppiah,doctors/dr.-tharmasewary-a~p-karuppiah,None,None,Medical Officer,MD,/malaysia/hospitals/tebrau/doctors
1,Dr. Zul Zarihi Bin Zainal,doctors/dr.-zul-zarihi-bin-zainal,None,None,Consultant Anesthesiologist,Doctor of Anesthesiology & Critical Care (UKM),/malaysia/hospitals/tebrau/doctors


In [39]:
def extract_area(link):
    print(link.split('/')[3])
    area = link.split('/')[3]
    return area
df_full["hospital"] =   df_full.apply(lambda x: extract_area(x["link"]), axis = 1)

cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
cheras
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bintulu
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rimau
bukit-rim

In [40]:
df_full.to_csv("columbia_list.csv")


In [42]:
df_full

,Name,Link,Speciality Title,Speciality Appointment,Designation,Qualification,link,hospital
0,Dr. Adam Lau Ee Chang,doctors/dr.-adam-lau-ee-chang,General Practice,https://malaysiaportal.columbiaasia.com/webcon...,Medical Officer,MD (CSMU),/malaysia/hospitals/cheras/doctors,cheras
1,Dr. Aiffa Syafira Binti Mohamad Amir Hamzah,doctors/dr.-aiffa-syafira-binti-mohamad-amir-h...,None,None,Medical Officer,MD (UPM),/malaysia/hospitals/cheras/doctors,cheras
2,Dr. Amir Bin Ramli @ Shaari,doctors/dr.-amir-bin-ramli-@-shaari,Internal Medicine,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Internal Medicine Physician,"MD (USM), MMed (Internal Medicine) (USM)",/malaysia/hospitals/cheras/doctors,cheras
3,Dr. Arun Arunasalam A/L Poraviappan,doctors/dr.-arun-arunasalam-a~l-poraviappan,Urology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Urologist,"M Surgery (UM), FRCS (Urology) (Glasgow), Fell...",/malaysia/hospitals/cheras/doctors,cheras
4,Dr. Azfar Rizal Ahmad,doctors/dr.-azfar-rizal-ahmad,Orthopedic,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Orthopedic and Arthroscopic Shoulde...,"MBBS (Australia), MS (Orthopedics) (UKM), Fell...",/malaysia/hospitals/cheras/doctors,cheras
...,...,...,...,...,...,...,...,...
4,Dr. Tee Shin Kuan,doctors/dr.-tee-shin-kuan,Internal Medicine and Nephrology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Internal Medicine Physician & Nephr...,"MBBS (UM), MRCP (UK)",/malaysia/hospitals/tebrau/doctors,tebrau
5,Dr. Tey Kwee Eng,doctors/dr.-tey-kwee-eng,Dermatology,https://malaysiaportal.columbiaasia.com/webcon...,Consultant Dermatologist (Visiting),"MD (UKM), MRCP (UK), Mmed (Internal Medicine)(...",/malaysia/hospitals/tebrau/doctors,tebrau
0,Dr. Tharmasewary A/P Karuppiah,doctors/dr.-tharmasewary-a~p-karuppiah,None,None,Medical Officer,MD,/malaysia/hospitals/tebrau/doctors,tebrau
1,Dr. Zul Zarihi Bin Zainal,doctors/dr.-zul-zarihi-bin-zainal,None,None,Consultant Anesthesiologist,Doctor of Anesthesiology & Critical Care (UKM),/malaysia/hospitals/tebrau/doctors,tebrau
